<h1> Building a Binary Sampling Matrix </h1>

In the following notebook, we build a binary sampling matrix using the observational measurements of various oceanic data compiled by Martiny & Flombaum (2020), [available at this link](https://www.bco-dmo.org/dataset/793451). 

Our sampling matrix is three dimensional, covering two dimensions of geospatial coordinates, and one dimension of time. On completion, our matrix will contain ones where observational data exists (spatially and temporally) and zero's where it doesn't, allowing us to sample from the output of model simulations accordingly. In order for this process to be effective, we must first ensure that time is treated approriately, and that measurement coordinates are mapped to the nearest simulation cell coordinates. 

We begin with cursory overview of the dataset, to get aquainted with its shape, size, and variables, and to ensure the datatypes are fit for our purposes. Once this has been completed, we perform a closer analysis of the data themselves, removing e.g. clear outliers or unphysical entries. Finally, we proceed to building our matrix. 


<h2> Data Overview </h2>

In [566]:
import xarray as xr

flom_ds = xr.open_dataset('/Users/leebardon/Desktop/stats_vs_gcm_data/flom_global_obs.netcdf')
flom_df = ds.to_dataframe()

In [567]:
flom_df.head(500)

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus,Pico_eukaryotes
unlimited,,,,,,,,,,,
0,1997.0,8.0,16.370,119.95,b'',b'',b'',0.5,b'',39770.00,b''
1,1997.0,9.0,16.348,119.93,b'',b'',b'',0.5,b'',15000.00,b''
2,1997.0,14.0,16.460,119.92,b'',b'',b'',0.5,b'',880.00,b''
3,1997.0,15.0,16.380,119.91,b'',b'',b'',0.5,b'',950.00,b''
4,1997.0,24.0,16.348,119.93,b'',b'',b'',0.5,b'',2730.00,b''
...,...,...,...,...,...,...,...,...,...,...,...
495,2003.0,162.0,41.492,339.98,b'5.27',b'0.37',b'13.511',101.0,b'304.52',832.95,b''
496,2003.0,162.0,41.492,339.98,b'5.52',b'0.4',b'13.399',126.0,b'30.33',464.04,b''
497,2003.0,162.0,41.492,339.98,b'6.25',b'0.45',b'13.224',150.8,b'88.06',680.99,b''


In [294]:
flom_ds.head()

<xarray.Dataset>
Dimensions:          (unlimited: 5)
Dimensions without coordinates: unlimited
Data variables:
    Year             (unlimited) float64 ...
    Day              (unlimited) float64 ...
    Latitude         (unlimited) float64 ...
    Longitude        (unlimited) float64 ...
    Nitrite_Nitrate  (unlimited) |S513 ...
    Phosphate        (unlimited) |S513 ...
    Temperature      (unlimited) |S513 ...
    Depth            (unlimited) float64 ...
    Prochlorococcus  (unlimited) |S513 ...
    Synechococcus    (unlimited) float64 ...
    Pico_eukaryotes  (unlimited) |S513 ...

In [403]:
flom_df.count()

Year               59554
Day                59554
Latitude           59554
Longitude          59554
Nitrite_Nitrate    59554
Phosphate          59554
Temperature        59554
Depth              59554
Prochlorococcus    59554
Synechococcus      59554
Pico_eukaryotes    59554
dtype: int64

In [425]:
flom_df.describe()

,Year,Day,Latitude,Longitude,Depth,Synechococcus
count,5.955400e+04,5.955400e+04,5.955400e+04,5.955400e+04,5.955400e+04,5.955400e+04
mean,1.054606e+34,1.176807e+35,8.353151e+34,4.720619e+34,3.232452e+35,2.389771e+36
std,3.240779e+35,1.076733e+36,9.087241e+35,6.843886e+35,1.765804e+36,4.255987e+36
min,1.987000e+03,1.000000e+00,-7.805000e+01,5.982900e-01,0.000000e+00,0.000000e+00
25%,1.995000e+03,1.160000e+02,2.000000e+00,1.668100e+02,1.800000e+01,6.460000e+02
50%,1.999000e+03,1.950000e+02,3.300000e+01,2.358200e+02,5.000000e+01,6.825300e+03
75%,2.003000e+03,2.940000e+02,4.341700e+01,3.004400e+02,1.000000e+02,1.329210e+05
max,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36


<h1> Cleaning the Data </h1> 

We immediately note that several of these columns are in a bytes-like format that we should decode, and coerce into a float datatype. Once we have processed these columns accordingly, we can proceed to examining the data in more detail. 


<h3> Decoding Bytes Objects </h3>

In [568]:
def decode_row(row):
    row = row.decode('utf-8')
    return row

decoded_rows = []
def decode_column(col):  
    for row in col:
        row = decode_row(row)
        decoded_rows.append(row)
        
decode_column(flom_df["Phosphate"])
flom_df["Phosphate"] = decoded_rows
decoded_rows.clear()

decode_column(flom_df["Nitrite_Nitrate"])
flom_df["Nitrite_Nitrate"] = decoded_rows
decoded_rows.clear()

decode_column(flom_df["Temperature"])
flom_df["Temperature"] = decoded_rows
decoded_rows.clear()

decode_column(flom_df["Prochlorococcus"])
flom_df["Prochlorococcus"] = decoded_rows
decoded_rows.clear()

decode_column(flom_df["Pico_eukaryotes"])
flom_df["Pico_eukaryotes"] = decoded_rows
decoded_rows.clear()

flom_df.head(500)

# NOTE: I'm still working on this method, and the one below - too much unnecessary repitition above, will return to this. 

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus,Pico_eukaryotes
unlimited,,,,,,,,,,,
0,1997.0,8.0,16.370,119.95,,,,0.5,,39770.00,
1,1997.0,9.0,16.348,119.93,,,,0.5,,15000.00,
2,1997.0,14.0,16.460,119.92,,,,0.5,,880.00,
3,1997.0,15.0,16.380,119.91,,,,0.5,,950.00,
4,1997.0,24.0,16.348,119.93,,,,0.5,,2730.00,
...,...,...,...,...,...,...,...,...,...,...,...
495,2003.0,162.0,41.492,339.98,5.27,0.37,13.511,101.0,304.52,832.95,
496,2003.0,162.0,41.492,339.98,5.52,0.4,13.399,126.0,30.33,464.04,
497,2003.0,162.0,41.492,339.98,6.25,0.45,13.224,150.8,88.06,680.99,


<h3> Coercing Strings to Floats </h3>

Decoding in this way returns the data as a string type, so we have a combination of empty and non-empty strings. Here, we will convert all non-empty strings to floats, and all empty strings to the NaN datatype, where NaN represents the lack of a measurement for a given value in that time/space.  

In [569]:
import numpy as np

converted_rows = []
def convert(col):
    for row in col:
        if row != "":
            row = float(row)
        else:
            row = np.nan
        converted_rows.append(row)

convert(flom_df["Phosphate"])
flom_df["Phosphate"] = converted_rows
converted_rows.clear()

convert(flom_df["Nitrite_Nitrate"])
flom_df["Nitrite_Nitrate"] = converted_rows
converted_rows.clear()

convert(flom_df["Temperature"])
flom_df["Temperature"] = converted_rows
converted_rows.clear()

convert(flom_df["Prochlorococcus"])
flom_df["Prochlorococcus"] = converted_rows
converted_rows.clear()

flom_df.head(500)

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus,Pico_eukaryotes
unlimited,,,,,,,,,,,
0,1997.0,8.0,16.370,119.95,NaN,NaN,NaN,0.5,NaN,39770.00,
1,1997.0,9.0,16.348,119.93,NaN,NaN,NaN,0.5,NaN,15000.00,
2,1997.0,14.0,16.460,119.92,NaN,NaN,NaN,0.5,NaN,880.00,
3,1997.0,15.0,16.380,119.91,NaN,NaN,NaN,0.5,NaN,950.00,
4,1997.0,24.0,16.348,119.93,NaN,NaN,NaN,0.5,NaN,2730.00,
...,...,...,...,...,...,...,...,...,...,...,...
495,2003.0,162.0,41.492,339.98,5.27,0.37,13.511,101.0,304.52,832.95,
496,2003.0,162.0,41.492,339.98,5.52,0.40,13.399,126.0,30.33,464.04,
497,2003.0,162.0,41.492,339.98,6.25,0.45,13.224,150.8,88.06,680.99,


In [570]:
flom_df.count()

Year               59554
Day                59554
Latitude           59554
Longitude          59554
Nitrite_Nitrate    38574
Phosphate          27769
Temperature        50096
Depth              59554
Prochlorococcus    42286
Synechococcus      59554
Pico_eukaryotes    59554
dtype: int64

Now that we have a working method, we can apply it to all the remaining columns that require decoding and conversion: 

<h1> Deeper Exploration </h1>  

In [571]:
flom_df.describe()

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus
count,5.955400e+04,5.955400e+04,5.955400e+04,5.955400e+04,38574.00000,27769.000000,50096.000000,5.955400e+04,4.228600e+04,5.955400e+04
mean,1.054606e+34,1.176807e+35,8.353151e+34,4.720619e+34,6.04649,0.844967,15.861593,3.232452e+35,4.162410e+04,2.389771e+36
std,3.240779e+35,1.076733e+36,9.087241e+35,6.843886e+35,8.53609,5.782614,9.226285,1.765804e+36,7.104746e+04,4.255987e+36
min,1.987000e+03,1.000000e+00,-7.805000e+01,5.982900e-01,-0.05600,-0.040000,-1.976000,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.995000e+03,1.160000e+02,2.000000e+00,1.668100e+02,0.12000,0.230000,8.550000,1.800000e+01,0.000000e+00,6.460000e+02
50%,1.999000e+03,1.950000e+02,3.300000e+01,2.358200e+02,2.59550,0.510000,16.364500,5.000000e+01,2.716000e+03,6.825300e+03
75%,2.003000e+03,2.940000e+02,4.341700e+01,3.004400e+02,8.26000,0.915134,24.048000,1.000000e+02,5.530600e+04,1.329210e+05
max,9.969210e+36,9.969210e+36,9.969210e+36,9.969210e+36,147.35000,367.000000,35.430000,9.969210e+36,1.820000e+06,9.969210e+36


We know from the above that we have ```59554``` measurements in total. However, given the values for the mean, standard deviation and max values, we can see that a number of these values seem suspcicious. 

For example, from the documentation, we know that the data were collected from 1987 to 2008, so the highest year value should be ```2.008e+03```, rather than ```9.969210e+36```. A good place to start would be to see how many rows have been given a year greater than 2008. 

In [572]:
greater_than_2008 = flom_df[flom_df['Year'] > 2.008e+03]
print(greater_than_2008)

                   Year           Day      Latitude     Longitude  \
unlimited                                                           
20102      9.969210e+36  9.969210e+36  9.969210e+36  7.866700e+00   
20103      9.969210e+36  9.969210e+36  9.969210e+36  7.866700e+00   
20104      9.969210e+36  9.969210e+36  9.969210e+36  7.866700e+00   
20105      9.969210e+36  9.969210e+36  9.969210e+36  7.866700e+00   
20106      9.969210e+36  9.969210e+36  9.969210e+36  7.866700e+00   
...                 ...           ...           ...           ...   
31049      9.969210e+36  9.969210e+36  9.969210e+36  9.969210e+36   
31050      9.969210e+36  9.969210e+36  9.969210e+36  9.969210e+36   
31051      9.969210e+36  9.969210e+36  9.969210e+36  9.969210e+36   
31052      9.969210e+36  9.969210e+36  9.969210e+36  9.969210e+36   
31053      9.969210e+36  9.969210e+36  9.969210e+36  9.969210e+36   

           Nitrite_Nitrate  Phosphate  Temperature  Depth  Prochlorococcus  \
unlimited               

We can see that 63 rows have been given the value ```9.969210e+36``` and that this persists across multiple columns. On further examination, it seems that this particular value is a default that signifies a missing value. Thus, we can go ahead and drop all rows with ```9.969210e+36``` as the year.

In [573]:
new_dataframe = flom_df.query('Year <= 2.008e+03') 
new_dataframe.describe()

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus
count,59491.000000,5.949100e+04,5.949100e+04,5.949100e+04,38511.000000,27769.000000,50096.000000,5.949100e+04,4.227800e+04,5.949100e+04
mean,1998.853087,1.072481e+35,7.306274e+34,3.870985e+34,6.056382,0.844967,15.861593,3.235875e+35,4.162873e+04,2.381745e+36
std,4.591027,1.028443e+36,8.503245e+35,6.200118e+35,8.539563,5.782614,9.226285,1.766708e+36,7.105337e+04,4.251083e+36
min,1987.000000,1.000000e+00,-7.805000e+01,5.982900e-01,-0.056000,-0.040000,-1.976000,0.000000e+00,0.000000e+00,0.000000e+00
25%,1995.000000,1.160000e+02,1.998800e+00,1.668050e+02,0.120000,0.230000,8.550000,1.800000e+01,0.000000e+00,6.444697e+02
50%,1999.000000,1.940000e+02,3.300000e+01,2.358200e+02,2.609000,0.510000,16.364500,5.000000e+01,2.712000e+03,6.800000e+03
75%,2003.000000,2.940000e+02,4.341700e+01,3.004400e+02,8.275000,0.915134,24.048000,1.000000e+02,5.532420e+04,1.280000e+05
max,2008.000000,9.969210e+36,9.969210e+36,9.969210e+36,147.350000,367.000000,35.430000,9.969210e+36,1.820000e+06,9.969210e+36


We can see that there are still missing values across various columns, even when the years with ```9.969210e+36``` values have been removed. 

In this case, we can go ahead and remove the 63 rows that have missing year values, because we can't use those. We will leave  the  rest in place for now, as some of the columns in that row may still have some useful data. 

<b> NOTE: </b> There are also some suspicious negative values for e.g. Temperature and Phosphate. We may need to return to this later, and remove or replace as appropriate. 

In [574]:
flom_df = new_dataframe

In [575]:
flom_df.count()

Year               59491
Day                59491
Latitude           59491
Longitude          59491
Nitrite_Nitrate    38511
Phosphate          27769
Temperature        50096
Depth              59491
Prochlorococcus    42278
Synechococcus      59491
Pico_eukaryotes    59491
dtype: int64

<h1> Data Processing </h1>

Now that we have cleaned the data - to at least a preliminary standard - we can begin to process it accordingly. To start, we will create a measurements data including latitude, longitude and time. 

Before doing do, for the purposes of ease and comparison, we should create a function that maps the time as months, numbered from ```1 to 252```, such that Jan 1987 would be month 1 and Dec 2008 would be month 252.

In the Flombaum dataset, measurements correspond to a given day, numbered ```1 to 366``` for each year. A reasonable approach might to create a new column that maps measurements days to a unique month, such that all measurements taken in Jan 1987 will be mapped to month 1, while all measurements taken in Jan 1988 will be mapped to month 13. 

Let's remove the missing values here too:

In [577]:
new_dataframe = flom_df.query('Day <= 9.96e+30') 
new_dataframe.describe()

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus
count,58851.000000,58851.000000,5.885100e+04,5.885100e+04,38149.000000,27762.000000,49990.000000,5.885100e+04,4.211400e+04,5.885100e+04
mean,1998.866340,196.805917,6.606501e+33,4.573731e+33,6.111545,0.845041,15.848734,3.249043e+35,4.151586e+04,2.322609e+36
std,4.588139,100.554015,2.565523e+35,2.134862e+35,8.560246,5.783340,9.225855,1.770178e+36,7.106104e+04,4.214305e+36
min,1987.000000,1.000000,-7.805000e+01,5.982900e-01,-0.056000,-0.040000,-1.976000,0.000000e+00,0.000000e+00,0.000000e+00
25%,1995.000000,115.000000,1.005200e+00,1.670000e+02,0.140000,0.230000,8.539250,1.800000e+01,0.000000e+00,6.291500e+02
50%,1999.000000,193.000000,3.291300e+01,2.358200e+02,2.677117,0.510000,16.350000,5.000000e+01,2.644000e+03,6.634500e+03
75%,2003.000000,293.000000,4.341700e+01,3.004400e+02,8.350000,0.915846,24.016750,1.000000e+02,5.481950e+04,1.060000e+05
max,2008.000000,366.000000,9.969210e+36,9.969210e+36,147.350000,367.000000,35.430000,9.969210e+36,1.820000e+06,9.969210e+36


This confirms that ```366``` is the maximum number of days.

In [578]:
flom_df = new_dataframe

In [579]:
flom_df["Year"].value_counts()

2001.0    5629
2002.0    5188
2004.0    4848
1999.0    4719
2003.0    4366
1992.0    4327
1996.0    3896
1994.0    3753
1995.0    3622
1998.0    3093
2005.0    2894
1997.0    2600
2000.0    2489
2006.0    2345
1993.0    1512
1991.0    1230
1989.0     833
2007.0     672
1990.0     462
1987.0     270
2008.0      63
1988.0      40
Name: Year, dtype: int64

<h1> Creating a Months' Column </h1>

In [743]:
import pandas as pd

flom_df["Month"] = 0
flom_df.head(500)

def assign_months(year):
    for i, row in flom_df.iterrows():
        val = flom_df.loc[i, "Year"]
        day = flom_df.loc[i, "Day"]
        if val != year:
            continue
        else:
            if  1.0 >= day < 31.0:
                flom_df.loc[i, "Month"] = months[0]
            elif 31.0 >= day < 60.0:
                flom_df.loc[i, "Month"] = months[1]
            elif 60.0 >= day < 91.0:
                flom_df.loc[i, "Month"] = months[2]
            elif 91.0 >= day < 121.0:
                flom_df.loc[i, "Month"] = months[3]
            elif 121.0 >= day < 152.0:
                flom_df.loc[i, "Month"] = months[4]
            elif 152.0 >= day < 182.0:
                flom_df.loc[i, "Month"] = months[5]
            elif 182.0 >= day < 213.0:
                flom_df.loc[i, "Month"] = months[6]
            elif 213.0 >= day < 244.0:
                flom_df.loc[i, "Month"] = months[7]
            elif 244.0 >= day < 275.0:
                flom_df.loc[i, "Month"] = months[8]
            elif 275.0 >= day < 306.0:
                flom_df.loc[i, "Month"] = months[9]
            elif 306.0 >= day < 336.0:
                flom_df.loc[i, "Month"] = months[10]
            else: 
                flom_df.loc[i, "Month"] = months[11]
                

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus,Pico_eukaryotes,Month
unlimited,,,,,,,,,,,,
0,1997.0,8.0,16.370,119.95,NaN,NaN,NaN,0.5,NaN,39770.00,,0
1,1997.0,9.0,16.348,119.93,NaN,NaN,NaN,0.5,NaN,15000.00,,0
2,1997.0,14.0,16.460,119.92,NaN,NaN,NaN,0.5,NaN,880.00,,0
3,1997.0,15.0,16.380,119.91,NaN,NaN,NaN,0.5,NaN,950.00,,0
4,1997.0,24.0,16.348,119.93,NaN,NaN,NaN,0.5,NaN,2730.00,,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2003.0,162.0,41.492,339.98,5.27,0.37,13.511,101.0,304.52,832.95,,0
496,2003.0,162.0,41.492,339.98,5.52,0.40,13.399,126.0,30.33,464.04,,0
497,2003.0,162.0,41.492,339.98,6.25,0.45,13.224,150.8,88.06,680.99,,0


We now have a column for ```Months``` where each value has initially been set to zero. The function ```assign_months()``` takes a single year as an argument, and iterates through the dataframe. Once it has detected a row with that year, it checks the day the measurement was taken, and assigns a month according to a specified range. 

<h2> Creating Years' List and Months' Vector </h2>

In [1285]:
# Create list of floats for each year

years_list = []
for i in np.arange(1987.0, 2009.0, 1):
    years_list.append(i)

# Create months vector that can be updated iteratively
months = pd.Series([1,2,3,4,5,6,7,8,9,10,11,12])

<h2> Assigning Months' based on Year and Day </h2>

In [745]:
for year in years_list:
    assign_months(year)
    months = months + 12

In [746]:
flom_df.describe()

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus,Month
count,58851.000000,58851.000000,5.885100e+04,5.885100e+04,38149.000000,27762.000000,49990.000000,5.885100e+04,4.211400e+04,5.885100e+04,58851.000000
mean,1998.866340,196.805917,6.606501e+33,4.573731e+33,6.111545,0.845041,15.848734,3.249043e+35,4.151586e+04,2.322609e+36,150.276920
std,4.588139,100.554015,2.565523e+35,2.134862e+35,8.560246,5.783340,9.225855,1.770178e+36,7.106104e+04,4.214305e+36,54.814934
min,1987.000000,1.000000,-7.805000e+01,5.982900e-01,-0.056000,-0.040000,-1.976000,0.000000e+00,0.000000e+00,0.000000e+00,10.000000
25%,1995.000000,115.000000,1.005200e+00,1.670000e+02,0.140000,0.230000,8.539250,1.800000e+01,0.000000e+00,6.291500e+02,106.000000
50%,1999.000000,193.000000,3.291300e+01,2.358200e+02,2.677117,0.510000,16.350000,5.000000e+01,2.644000e+03,6.634500e+03,154.000000
75%,2003.000000,293.000000,4.341700e+01,3.004400e+02,8.350000,0.915846,24.016750,1.000000e+02,5.481950e+04,1.060000e+05,197.000000
max,2008.000000,366.000000,9.969210e+36,9.969210e+36,147.350000,367.000000,35.430000,9.969210e+36,1.820000e+06,9.969210e+36,264.000000


In [747]:
flom_df.head(500)

,Year,Day,Latitude,Longitude,Nitrite_Nitrate,Phosphate,Temperature,Depth,Prochlorococcus,Synechococcus,Pico_eukaryotes,Month
unlimited,,,,,,,,,,,,
0,1997.0,8.0,16.370,119.95,NaN,NaN,NaN,0.5,NaN,39770.00,,122
1,1997.0,9.0,16.348,119.93,NaN,NaN,NaN,0.5,NaN,15000.00,,122
2,1997.0,14.0,16.460,119.92,NaN,NaN,NaN,0.5,NaN,880.00,,122
3,1997.0,15.0,16.380,119.91,NaN,NaN,NaN,0.5,NaN,950.00,,122
4,1997.0,24.0,16.348,119.93,NaN,NaN,NaN,0.5,NaN,2730.00,,122
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2003.0,162.0,41.492,339.98,5.27,0.37,13.511,101.0,304.52,832.95,,199
496,2003.0,162.0,41.492,339.98,5.52,0.40,13.399,126.0,30.33,464.04,,199
497,2003.0,162.0,41.492,339.98,6.25,0.45,13.224,150.8,88.06,680.99,,199


<h1> Matching Observations with Gridded Model </h1>

<h2> Algorithmic Overview </h2> 

<ol>

1. Read in a vector ```X```, ```Y``` and ```T``` of measurements' longitudes, latitudes, and time, respectively (all 1D vectors of size N). In this case, time, T, has been started as months ranging from 1 to 264. 
    

2. Read in a vector ```x``` (1D, size n_x =144) of grid cell center longitudes, a vector ```y``` (1D, size n_y =90) of grid cell center latitudes, and a vector ```t``` of months (1D, size n_t =12(months)*22(years from Jan 1987 to Jan 2009)).
    
    
3. Create a 3D matrix of zeros I (size n_x, n_y, n_t) 
    

4. Create a loop where j goes from 1 to N. For each measurement j, find the 3D index (ix,iy,it) where the absolute value of x(ix) is closest to X(j), the absolute value of y(iy) is closest to Y(j), and then also let it = T(j). Set I(ix,iy,it) = 1.

</ol>


<h3> Part A </h3>

In [1199]:
X = flom_df["Longitude"][:]
Y = flom_df["Latitude"][:]
T = flom_df["Month"][:]

print(" Measurements Lat: ", X.shape, "\n", "Measurements Lon: ",  Y.shape, "\n",  "Measurements Time: ", T.shape)

 Measurements Lat:  (58851,) 
 Measurements Lon:  (58851,) 
 Measurements Time:  (58851,)


Before we go any further, let's make sure that all rows containing the default missing value ```9.969209968386869e+36``` have been removed, as these could cause problems with the following calculations. 

In [1083]:
def count_nums(series, max_value):
    total = 0
    for e in series:
        if e > max_value:
            total += 1
    print("Total out-of-bounds = ", total)

In [1084]:
count_nums(X, 360)

Total out-of-bounds =  27


In [1158]:
count_nums(Y, 90)

Total out-of-bounds =  39


So we have 39 values that are out of bounds for ```Y```, and 27 for ```X```. Since we want all columns ```X```, ```Y``` and ```T``` to be equal in size, we'll concatenate the three vectors into a matrix, and remove all rows where both ```X``` and ```Y``` have out-of-bounds values:

In [1165]:
test_matrix = np.vstack([X,Y,T]).T
test_matrix.shape

(58851, 3)

First, let's check is Y has any values greater than 90, but less than 360. If not, then we can go ahead and remove all values greater than 360 from the matrix. 

In [1166]:
count_nums(Y, 360)

Total out-of-bounds =  39


Great! Moving on:

In [1174]:
out_bounds = np.where(np.any(test_matrix > 360, axis=1))
print(out_bounds[0])
len(out_bounds[0])

[18652 18653 18654 18655 18656 18657 18658 18659 18660 18661 18662 18663
 39290 39291 39292 39293 39294 39295 39296 39297 39298 39299 39300 39521
 40069 40187 40188 40189 40190 40191 40192 40193 40194 40254 40256 40408
 40417 40517 40599]


39

This seems reasonable - let's remove those:

In [1175]:
cleaned_matrix = np.delete(test_matrix, out_bounds[0], axis=0)

In [1176]:
cleaned_matrix.shape

(58812, 3)

In [1201]:
X = cleaned_matrix[:, 0]
Y = cleaned_matrix[:, 1]
T = cleaned_matrix[:, 2]

In [1204]:
print(" Measurements Lat: ", X.shape, "\n", "Measurements Lon: ",  Y.shape, "\n",  "Measurements Time: ", T.shape)

 Measurements Lat:  (58812,) 
 Measurements Lon:  (58812,) 
 Measurements Time:  (58812,)


<h3> Part B </h3>

In [790]:
from netCDF4 import Dataset 

grid = Dataset('grid_igsm.nc', 'r')

In [1255]:
x = grid.variables["X"][:]
y = grid.variables["Y"][:]
t = np.arange(1, 265, 1)

In [938]:
print(" Grid Lat: ", x.shape, "\n", "Grid Lon: ",  y.shape, "\n",  "Grid Time: ", t.shape)

 Grid Lat:  (144,) 
 Grid Lon:  (90,) 
 Grid Time:  (264,)


<h3> Part C </h3>

In [1222]:
I = np.zeros(shape=(144, 90, 265))
I.shape

(144, 90, 265)

<h3> Part D </h3>

In [934]:
flom_df.shape

(58851, 12)

In [1210]:
# Small scale sanity check:

for j in range(0, 5):
    ix = abs(X[j]-x).argmin()
    iy = abs(Y[j]-y).argmin()
    print(ix, iy)
           
print( X[:5], "\n", Y[:5], "\n")
print(" Model grid x[47] = ", x[47], "\n", "Model grid y[53] = ", y[53], "\n")
print(" x[45:49]", x[45:50], "\n", "y[51:55]", y[51:56])

47 53
47 53
47 53
47 53
47 53
[119.95 119.93 119.92 119.91 119.93] 
 [16.37  16.348 16.46  16.38  16.348] 

 Model grid x[47] =  118.75 
 Model grid y[53] =  16.0 

 x[45:49] [113.75 116.25 118.75 121.25 123.75] 
 y[51:55] [12. 14. 16. 18. 20.]


We can see from the small scale test that we appear to be identifying the correct nearest indices for between grid cell coordinates and measurement coordinates in space. 

Now, we can perform the full computation for each measurement. We also include the month for each measurement (already calculated earlier), and, for each ix, iy and it, we change the relevant 3D index of our matrix of zeros, to a one, resulting in a binary matrix where each one represent a space and temporal coordinate where we an accompanying observational measurement. 

In [1224]:
for j in range(0, 58812):
    ix = abs(X[j]-x).argmin()
    iy = abs(Y[j]-y).argmin()
    it = int(T[j])
    I[ix,iy,it] = 1

In [1283]:
measurements = np.where(np.any(I > 0, axis=1))
print("Total Space-Time Volume = ", 144*90*265, "\n", "Volume of Measurement Space = ", len(measurements[0]))

Total Space-Time Volume =  3434400 
 Volume of Measurement Space =  2624
